In [ ]:
!pip install auto-gptq optimum accelerate transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.8/425.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!nvidia-smi

Mon Aug 11 06:49:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from auto_gptq import AutoGPTQForCausalLM
from transformers import AutoTokenizer

model_id = "TheBloke/Llama-2-7B-Chat-GPTQ"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(
    model_id,
    device_map="auto",              # ✅ FIX: Don't use device="cuda"
    use_safetensors=True,
    trust_remote_code=True,
    use_triton=False                # Optional, for Colab stability
)

config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

WARNING - Exllamav2 kernel is not installed, reset disable_exllamav2 to True. This may because you installed auto_gptq using a pre-build wheel on Windows, in which exllama_kernels are not compiled. To use exllama_kernels to further speedup inference, you can re-install auto_gptq from source.
WARNING - CUDA kernels for auto_gptq are not installed, this will result in very slow inference speed. This may because:
1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.
1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads a

quantize_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

INFO - The layer lm_head is not quantized.
INFO:auto_gptq.modeling._base:The layer lm_head is not quantized.


In [ ]:
# Define prompt
prompt = "Explain the difference between GPTQ and AWQ in simple terms."

# Tokenize prompt
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

In [ ]:
# Generate output
outputs = model.generate(
    **inputs,
    max_new_tokens=64,
    do_sample=True,
    top_p=0.95,
    temperature=0.7
)

In [ ]:
# Decode output
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
print(response)

Explain the difference between GPTQ and AWQ in simple terms.
GPTQ and AWQ are two different types of questions that are commonly used in various assessments, including job interviews, tests, and surveys. The main difference between GPTQ and AWQ is the level of complexity and the type of information required to answer the question.
GPT


In [ ]:
# text_gen = pipeline("text-generation", model=model, tokenizer=tokenizer)
# output = text_gen(prompt, max_new_tokens=256, do_sample=True)
# print(output[0]['generated_text'])

In [ ]:
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
from transformers import AutoTokenizer

model_id = "tiiuae/falcon-rw-1b"

quant_config = BaseQuantizeConfig(
    bits=4,
    group_size=128,
    desc_act=False
)

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

# Fix padding token issue
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoGPTQForCausalLM.from_pretrained(
    model_id,
    quantize_config=quant_config,
    trust_remote_code=True
)

# Calibration texts
example_texts = [
    "What is AI?",
    "Explain the purpose of Falcon model.",
    "List 3 benefits of GPTQ.",
    "How does quantization work in LLMs?",
    "Give an example of a large language model.",
]

# Tokenize
examples = tokenizer(example_texts, return_tensors="pt", padding=True, truncation=True)
examples = [ {k: v[i] for k, v in examples.items()} for i in range(len(example_texts)) ]

tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

WARNING - can't get model's sequence length from model config, will set to 4096.
INFO - Start quantizing layer 1/24
INFO:auto_gptq.modeling._base:Start quantizing layer 1/24
INFO - Quantizing self_attention.query_key_value in layer 1/24...
INFO:auto_gptq.modeling._base:Quantizing self_attention.query_key_value in layer 1/24...
INFO - Quantizing self_attention.dense in layer 1/24...
INFO:auto_gptq.modeling._base:Quantizing self_attention.dense in layer 1/24...
INFO - Quantizing mlp.dense_h_to_4h in layer 1/24...
INFO:auto_gptq.modeling._base:Quantizing mlp.dense_h_to_4h in layer 1/24...
INFO - Quantizing mlp.dense_4h_to_h in layer 1/24...
INFO:auto_gptq.modeling._base:Quantizing mlp.dense_4h_to_h in layer 1/24...
INFO - Start quantizing layer 2/24
INFO:auto_gptq.modeling._base:Start quantizing layer 2/24
INFO - Quantizing self_attention.query_key_value in layer 2/24...
INFO:auto_gptq.modeling._base:Quantizing self_attention.query_key_value in layer 2/24...
INFO - Quantizing self_attenti

('falcon-rw-1b-gptq/tokenizer_config.json',
 'falcon-rw-1b-gptq/special_tokens_map.json',
 'falcon-rw-1b-gptq/vocab.json',
 'falcon-rw-1b-gptq/merges.txt',
 'falcon-rw-1b-gptq/added_tokens.json',
 'falcon-rw-1b-gptq/tokenizer.json')

In [ ]:
output_dir = "falcon-rw-1b-gptq"
model.save_quantized(output_dir, use_safetensors=True)
tokenizer.save_pretrained(output_dir)
# IMPORTANT: save config too
from transformers import AutoConfig
AutoConfig.from_pretrained(model_id, trust_remote_code=True).save_pretrained(output_dir)

In [ ]:
!pip install -U "transformers>=4.37" accelerate
!pip install -U pip setuptools wheel ninja
!pip install -U "torch" "torchvision" --index-url https://download.pytorch.org/whl/cu121
!pip install -U transformers accelerate
!pip install "auto-gptq>=0.7.1" --no-build-isolation
# (runtime restart recommended)

In [ ]:
from auto_gptq import AutoGPTQForCausalLM
from transformers import AutoTokenizer
model_path = "falcon-rw-1b-gptq"
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)
model = AutoGPTQForCausalLM.from_quantized(
    model_path,
    device_map="auto",
    trust_remote_code=False,   # ⬅️ avoids modeling_falcon.py requirement
    use_safetensors=True,
    use_triton=False,
    disable_exllamav2=True     # ⬅️ suppress the exllama warning
)

WARNING - Exllama kernel is not installed, reset disable_exllama to True. This may because you installed auto_gptq using a pre-build wheel on Windows, in which exllama_kernels are not compiled. To use exllama_kernels to further speedup inference, you can re-install auto_gptq from source.
WARNING - CUDA kernels for auto_gptq are not installed, this will result in very slow inference speed. This may because:
1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.
1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.
WARNING - ignoring unknown parameter in quantize_config.json: quant_method.
INFO - The layer lm_head is not quantized.
INFO:auto_gptq.modeling._base:The layer lm_head is not quan

In [ ]:
import torch
import time

# Evaluation prompts
prompts = [
    "What is the capital of India?",
    "Explain how quantization improves inference.",
    "What is the Falcon model used for?",
    "List 3 applications of LLMs.",
    "Define GPTQ in simple terms.",
]

# Evaluation loop
total_time = 0.0
all_outputs = []

for i, prompt in enumerate(prompts):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Start timing
    start = time.time()

    outputs = model.generate(
        **inputs,
        max_new_tokens=64,
        do_sample=False  # deterministic output for eval
    )

    # End timing
    end = time.time()
    inference_time = end - start
    total_time += inference_time

    # Decode response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    all_outputs.append((prompt, response, round(inference_time, 3)))

# Print all results
for i, (prompt, response, t) in enumerate(all_outputs):
    print(f"\n Prompt {i+1}: {prompt}")
    print(f" Response: {response}")
    print(f" Inference Time: {t} seconds")

# Average time
avg_time = total_time / len(prompts)
print(f"\n Average Inference Time: {round(avg_time, 3)} seconds per prompt")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



 Prompt 1: What is the capital of India?
 Response: What is the capital of India?
The capital of India is New Delhi.
What is the capital of India?
The capital of India is New Delhi.
What is the capital of India?
The capital of India is New Delhi.
What is the capital of India?
The capital of India is New Delhi.
What is the
 Inference Time: 9.378 seconds

 Prompt 2: Explain how quantization improves inference.
 Response: Explain how quantization improves inference.
Quantization is a technique that allows us to reduce the size of a signal by reducing the number of bits.
The number of bits in a signal is called the quantization level.
The quantization level is the smallest number of bits that can be used to represent a signal.
The quantization level is
 Inference Time: 9.143 seconds

 Prompt 3: What is the Falcon model used for?
 Response: What is the Falcon model used for?
The Falcon model is used to determine the amount of money that a person has to pay for a particular service. It is u

In [ ]:
!pip -q install -U huggingface_hub
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from huggingface_hub import whoami
whoami()

In [ ]:
from huggingface_hub import upload_folder
upload_folder(
    repo_id="sunny199/quantized-falcon-rw-1b",
    folder_path="./falcon-rw-1b-gptq",
    commit_message="Upload GPTQ quantized Falcon model"
)

In [ ]:
import os, glob, json
path = "falcon-rw-1b-gptq"
print(os.listdir(path))  # -> quantize_config.json, tokenizer files, gptq_model-4bit-*.safetensors

with open(f"{path}/quantize_config.json") as f:
    print(json.load(f))  # -> {"bits": 4, "group_size": 128, "desc_act": false, ...}
print(glob.glob(f"{path}/gptq_model-*.safetensors"))


['quantize_config.json', 'config.json', 'vocab.json', 'tokenizer.json', 'merges.txt', 'configuration_falcon.py', 'tokenizer_config.json', 'special_tokens_map.json', 'gptq_model-4bit-128g.safetensors']
{'bits': 4, 'group_size': 128, 'damp_percent': 0.01, 'desc_act': False, 'static_groups': False, 'sym': True, 'true_sequential': True, 'model_name_or_path': 'falcon-rw-1b-gptq', 'model_file_base_name': 'gptq_model-4bit-128g', 'is_marlin_format': False, 'quant_method': 'gptq'}
['falcon-rw-1b-gptq/gptq_model-4bit-128g.safetensors']


# AWQ Quantization for LLM